# 1. Import packages

In [18]:
import copy
import os
import pandas as pd

from tld import get_fld

# 2. Read a TXT

## 2.1. Change to input directory

In [2]:
print(os.getcwd())
input_path_s = "/input/"
input_path_s = os.getcwd() + input_path_s
output_path_s = "/output/"
output_path_s = os.getcwd() + output_path_s
os.chdir(input_path_s)
print(os.getcwd())

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task18_tracker_domain_tracker
/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task18_tracker_domain_tracker/input


## 2.2. Read additional-trackers-all-sources.txt

In [3]:
tracker_df = pd.read_csv("additional-trackers-all-sources.txt", header=None)
print(tracker_df.shape)

(273, 1)


In [4]:
tracker_l = list(tracker_df[0])
del tracker_df
print(len(tracker_l))
print(tracker_l[:10])

273
['fwmrm.net', 'ad.gt', 'teads.tv', 'dotomi.com', 'ads-twitter.com', 'intelligenceadx.com', 'strossle.com', 'adskeeper.com', 'pagefair.net', 'crm4d.com']


## 2.3. Read measurement_ids_kicho_tracker.txt

In [5]:
target_domain_df = pd.read_csv("measurement_ids_kicho_tracker.txt", header=None)
print(target_domain_df.shape)

(494261, 6)


In [6]:
target_domain_df

,0,1,2,3,4,5
0,0,de,39835,www.instagram.com,42506263.0,False
1,1,de,39835,twitter.com,42506264.0,True
2,2,de,39835,s0.2mdn.net,42506265.0,False
3,3,de,39835,www.clarity.ms,42506266.0,False
4,4,de,39835,www.filmix.ac,42506267.0,False
...,...,...,...,...,...,...
494256,494256,cz,44489,cdn.speedcurve.com,43832951.0,False
494257,494257,cz,44489,img.kurzy.cz,43832953.0,False
494258,494258,cz,44489,report.prozeny.cz,43832954.0,False
494259,494259,cz,44489,pro.mapy.cz,43832955.0,False


In [7]:
target_domain_df.columns = ["seq", "country", "asn", "target_domain", "measurement_id", "is_tracker"]

In [8]:
target_domain_df.head(5)

,seq,country,asn,target_domain,measurement_id,is_tracker
0,0,de,39835,www.instagram.com,42506263.0,False
1,1,de,39835,twitter.com,42506264.0,True
2,2,de,39835,s0.2mdn.net,42506265.0,False
3,3,de,39835,www.clarity.ms,42506266.0,False
4,4,de,39835,www.filmix.ac,42506267.0,False


## 2.4. tld_l and two_letter_tld_set

In [40]:
tld_l = list(pd.read_csv("tld-list-basic.txt", header=None)[0])

In [41]:
two_letter_tld_set = set()

# I am pretty sure there is a way to parse and read at the same time, instead of reading all at once.
for tld in tld_l:
    if len(tld) == 2:
        two_letter_tld_set.add("." + tld)

two_letter_tld_set -= set([".ai", ".io"])
print(len(two_letter_tld_set))
print(two_letter_tld_set)

317
{'.um', '.ag', '.nc', '.ml', '.bg', '.台灣', '.jo', '.af', '.sc', '.tl', '.st', '.cr', '.cy', '.nz', '.mk', '.ar', '.gs', '.hu', '.be', '.dk', '.cg', '.jm', '.mm', '.bi', '.am', '.al', '.hk', '.je', '.pf', '.ht', '.co', '.la', '.vu', '.au', '.移动', '.世界', '.tk', '.cv', '.健康', '.cx', '.cz', '.sz', '.信息', '.pe', '.公益', '.gd', '.td', '.mr', '.lt', '.mz', '.ad', '.珠宝', '.vg', '.mh', '.bz', '.cf', '.es', '.qa', '.购物', '.ne', '.aq', '.ge', '.kw', '.cu', '.嘉里', '.ao', '.ke', '.測試', '.ls', '.mn', '.jp', '.广东', '.pk', '.商城', '.zw', '.rw', '.br', '.lu', '.网店', '.测试', '.dj', '.eu', '.八卦', '.通販', '.gn', '.澳門', '.gh', '.佛山', '.by', '.გე', '.pa', '.ms', '.ec', '.su', '.pt', '.cd', '.bn', '.中国', '.fr', '.mq', '.tz', '.pn', '.ie', '.mc', '.no', '.sg', '.娱乐', '.tw', '.集团', '.lr', '.tc', '.kh', '.eg', '.cw', '.sr', '.md', '.gf', '.ki', '.tj', '.ευ', '.zm', '.py', '.pl', '.时尚', '.mg', '.餐厅', '.vi', '.tp', '.nl', '.mo', '.sb', '.im', '.vc', '.mf', '.机构', '.li', '.kg', '.닷넷', '.lb', '.bf', '.ma', '.手机', '

# 3. Validate the current input files

## 3.1 All FLD in tracker_l?

In [9]:
for tracker in tracker_l:
    if tracker.count(".") > 1:
        print(tracker)

adskeeper.co.uk
anm.co.uk
dmgmediaprivacy.co.uk
google.com.sv
google.com.tr


## 3.2. How many true in the last column in target_domain_df?

In [10]:
target_domain_df["is_tracker"].value_counts()

False    401228
True      93033
Name: is_tracker, dtype: int64

In [11]:
target_domain_df["is_tracker"].value_counts(normalize=True)

False    0.811774
True     0.188226
Name: is_tracker, dtype: float64

## 3.3. Is tracker_l unique?

In [12]:
print(len(set(tracker_l)) == len(tracker_l))
print(len(set(tracker_l)) - len(tracker_l))

False
-1


### 3.3.1. Use tracker_set

In [13]:
tracker_set = set(tracker_l)

# 4. FLD column

## 4.1. Initial target_domain_fld_l

In [16]:
target_domain_fld_l = []

for target_domain in target_domain_df["target_domain"]:
    try:
        target_domain_fld_l.append(get_fld(target_domain, fix_protocol=True))
    except:
        print(target_domain)
        target_domain_fld_l.append(target_domain)
        
print(len(target_domain_fld_l) == len(target_domain_df))
print(target_domain_fld_l[:20])

172.31.2.38
192.168.8.1
192.168.8.1
188.191.200.4
192.168.8.1
192.168.8.1
192.168.8.1
192.168.8.1
192.168.8.1
192.168.8.1
192.168.8.1
True
['instagram.com', 'twitter.com', '2mdn.net', 'clarity.ms', 'filmix.ac', 'quantserve.com', 'gls-pakete.de', 'telewebion.com', 'n-tv.de', 'media.net', 'wt-safetag.com', 'googletagservices.com', 'relap.info', 'outbrain.com', 'wetter.com', 'amazonaws.com', 'akamaihd.net', 'aparat.com', 'vk.com', 'ssl-images-amazon.com']


## 4.2. get_fld_by_reversing()

In [51]:
def get_fld_by_reversing(url):
    if url[-3:] not in two_letter_tld_set and not url[-1].isnumeric():
        reversed_url = url[::-1]
        first_dot_i = reversed_url.index(".")
        second_dot_i = reversed_url[first_dot_i+1:].index(".")
        return url[-(first_dot_i+1+second_dot_i):]
    # else:
    #     return url

## 4.3. fld_to_clean_set

In [55]:
fld_to_clean_set = set()

for fld in target_domain_fld_l:
    if fld.count(".") > 1:
        fld_to_clean_set.add(get_fld_by_reversing(fld))

print(len(fld_to_clean_set))
print(fld_to_clean_set)

11
{'googleapis.com', 'cloudfunctions.net', 'appspot.com', 'amazonaws.com', 'cloudfront.net', 'cdn77.org', 'fastly.net', 'azurewebsites.net', 'run.app', 'github.io', None}


In [56]:
fld_to_clean_set.remove(None)
print(fld_to_clean_set)

{'googleapis.com', 'cloudfunctions.net', 'appspot.com', 'amazonaws.com', 'cloudfront.net', 'cdn77.org', 'fastly.net', 'azurewebsites.net', 'run.app', 'github.io'}


## 4.4. Clean target_domain_fld_l

In [59]:
dirty_target_domain_fld_l = copy.deepcopy(target_domain_fld_l)

In [60]:
count = 0
for i, fld in enumerate(target_domain_fld_l):
    for clean_fld in fld_to_clean_set:
        if clean_fld in fld:
            target_domain_fld_l[i] = clean_fld
            count += 1
            
print(count)

8593


## 4.5. Add a column: target_domain_fld

In [73]:
target_domain_df.shape

(494261, 6)

In [74]:
target_domain_6col_df = copy.deepcopy(target_domain_df)

In [75]:
target_domain_df["target_domain_fld"] = target_domain_fld_l
print(target_domain_df.shape)

(494261, 7)


In [76]:
target_domain_df

,seq,country,asn,target_domain,measurement_id,is_tracker,target_domain_fld
0,0,de,39835,www.instagram.com,42506263.0,False,instagram.com
1,1,de,39835,twitter.com,42506264.0,True,twitter.com
2,2,de,39835,s0.2mdn.net,42506265.0,False,2mdn.net
3,3,de,39835,www.clarity.ms,42506266.0,False,clarity.ms
4,4,de,39835,www.filmix.ac,42506267.0,False,filmix.ac
...,...,...,...,...,...,...,...
494256,494256,cz,44489,cdn.speedcurve.com,43832951.0,False,speedcurve.com
494257,494257,cz,44489,img.kurzy.cz,43832953.0,False,kurzy.cz
494258,494258,cz,44489,report.prozeny.cz,43832954.0,False,prozeny.cz
494259,494259,cz,44489,pro.mapy.cz,43832955.0,False,mapy.cz


# 5. Be consistent in tracker marking in target_domain_df

## 5.1. is_tracker_fld_set

In [78]:
target_domain_df = target_domain_df.sort_values(by=['target_domain_fld'], ignore_index=True)
target_domain_df

,seq,country,asn,target_domain,measurement_id,is_tracker,target_domain_fld
0,189728,se,2833,www.1177.se,43345322.0,False,1177.se
1,186574,se,2119,www.1177.se,43340394.0,False,1177.se
2,181448,se,42303,www.1177.se,43332830.0,False,1177.se
3,181040,se,35100,www.1177.se,43332284.0,False,1177.se
4,179460,se,51929,www.1177.se,43329833.0,False,1177.se
...,...,...,...,...,...,...,...
494256,259407,fr,8362,a.zxcvads.com,43474978.0,False,zxcvads.com
494257,272167,fr,51207,a.zxcvads.com,43492070.0,False,zxcvads.com
494258,260201,fr,8218,a.zxcvads.com,43476138.0,False,zxcvads.com
494259,270236,fr,34970,a.zxcvads.com,43489380.0,False,zxcvads.com


In [79]:
is_tracker_fld_set = set()

for i in range(len(target_domain_df)):
    if target_domain_df["is_tracker"][i]:
        is_tracker_fld_set.add(target_domain_df["target_domain_fld"][i])
        
print(len(is_tracker_fld_set))
print(is_tracker_fld_set)

284
{'welt.de', 'cognitivlabs.com', 'naszemiasto.pl', 'outbrain.com', 'yle.fi', 'chip.de', 'epimg.net', 'facebook.net', 'bamgrid.com', 'mgid.com', 'gazeta.pl', 'licdn.com', '24.hu', 'google.com.ua', 'taboola.com', 'googleapis.com', 'list-manage.com', 'reevoo.com', 'onetrust.com', 'linkedin.com', 'baqend.com', 'siteimproveanalytics.io', 'ispot.tv', 'telekom.de', 'redditmedia.com', 'pix.pub', 'viafoura.co', 'm6r.eu', 'mirror.co.uk', 'pswec.com', 'focus.de', 'liveperson.net', 'parsely.com', 'cxense.com', 'pinterest.com', 'typicalairplane.com', 'azure.com', 'sentry-cdn.com', 'datadome.co', 'sensic.net', 'dailymotion.com', 'gedidigital.it', 'bbc.co.uk', 'vulcan.net.pl', 'stroeerdigitalgroup.de', 'mediaset.it', 'a2z.com', 'seznam.cz', 'screen9.com', 'placed.com', 'ok.ru', 'mrf.io', 'nelonenmedia.fi', 'optimizely.com', 'pghub.io', 'sascdn.com', 'google.com', 'stackadapt.com', 'suomi.fi', 't-online.de', 'lijit.com', 'scottishdailyexpress.co.uk', 'samsung.com', 'rumble.com', 'sanoma-sndp.fi', '

## 5.2. Update is_tracker

In [80]:
target_domain_df["is_tracker"].value_counts()

False    401228
True      93033
Name: is_tracker, dtype: int64

In [81]:
for i in range(len(target_domain_df)):
    if target_domain_df["target_domain_fld"][i] in is_tracker_fld_set:
        target_domain_df.loc[i, "is_tracker"] = True

In [82]:
target_domain_df["is_tracker"].value_counts()

False    300647
True     193614
Name: is_tracker, dtype: int64

In [83]:
target_domain_df.shape

(494261, 7)

In [84]:
target_domain_7col_df = copy.deepcopy(target_domain_df)

# 6. is_more_tracker_l

In [89]:
is_more_tracker_l = [False] * len(target_domain_df)

for i, fld in enumerate(target_domain_df["target_domain_fld"]):
    if fld in tracker_set:
        is_more_tracker_l[i] = True

print(len(is_more_tracker_l) == len(target_domain_df))
print(pd.DataFrame(is_more_tracker_l).value_counts())
print(is_more_tracker_l[:20])

True
False    436656
True      57605
dtype: int64
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


# 6. is_new_tracker_l

## 6.1. is_new_tracker_l

In [97]:
is_new_tracker_l = [None] * len(target_domain_df)
new_tracker_index_l = []

for index, original_tracker in enumerate(target_domain_df["is_tracker"]):
    if original_tracker is True:
        is_new_tracker_l[index] = True
    elif original_tracker is False and is_more_tracker_l[index] is True:
        new_tracker_index_l.append(index)
        is_new_tracker_l[index] = True
    else:
        is_new_tracker_l[index] = False

print(len(is_new_tracker_l) == len(target_domain_df))
print(pd.DataFrame(is_new_tracker_l).value_counts())
print(is_new_tracker_l[:20])

True
True     251066
False    243195
dtype: int64
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [98]:
list(target_domain_df["is_tracker"]).count(True) < is_new_tracker_l.count(True)

True

## 6.2. List of newly discovered target_domain

In [99]:
print(len(new_tracker_index_l))
print(new_tracker_index_l[:10])

57452
[777, 778, 779, 780, 781, 782, 783, 784, 785, 786]


In [100]:
# Is any of the pre-existing tracker True? There should be no True.
# Thus, False would be an output from this any()
any(target_domain_df.iloc[new_tracker_index_l, target_domain_df.columns == "is_tracker"].squeeze())

False

In [101]:
new_tracker_target_domain_df = target_domain_df.iloc[new_tracker_index_l, target_domain_df.columns == "target_domain"]
new_tracker_target_domain_df

,target_domain
777,sync.1rx.io
778,sync.1rx.io
779,sync.1rx.io
780,sync.1rx.io
781,sync.1rx.io
...,...
494256,a.zxcvads.com
494257,a.zxcvads.com
494258,a.zxcvads.com
494259,a.zxcvads.com


# 7. Output dfs

## 7.1. new_target_domain_df

In [102]:
target_domain_df

,seq,country,asn,target_domain,measurement_id,is_tracker,target_domain_fld
0,189728,se,2833,www.1177.se,43345322.0,False,1177.se
1,186574,se,2119,www.1177.se,43340394.0,False,1177.se
2,181448,se,42303,www.1177.se,43332830.0,False,1177.se
3,181040,se,35100,www.1177.se,43332284.0,False,1177.se
4,179460,se,51929,www.1177.se,43329833.0,False,1177.se
...,...,...,...,...,...,...,...
494256,259407,fr,8362,a.zxcvads.com,43474978.0,False,zxcvads.com
494257,272167,fr,51207,a.zxcvads.com,43492070.0,False,zxcvads.com
494258,260201,fr,8218,a.zxcvads.com,43476138.0,False,zxcvads.com
494259,270236,fr,34970,a.zxcvads.com,43489380.0,False,zxcvads.com


In [103]:
new_target_domain_df = target_domain_df.drop(["is_tracker", "target_domain_fld"], axis=1)
new_target_domain_df

,seq,country,asn,target_domain,measurement_id
0,189728,se,2833,www.1177.se,43345322.0
1,186574,se,2119,www.1177.se,43340394.0
2,181448,se,42303,www.1177.se,43332830.0
3,181040,se,35100,www.1177.se,43332284.0
4,179460,se,51929,www.1177.se,43329833.0
...,...,...,...,...,...
494256,259407,fr,8362,a.zxcvads.com,43474978.0
494257,272167,fr,51207,a.zxcvads.com,43492070.0
494258,260201,fr,8218,a.zxcvads.com,43476138.0
494259,270236,fr,34970,a.zxcvads.com,43489380.0


## 7.2. Add a column: is_new_tracker

In [104]:
new_target_domain_df["is_new_tracker"] = is_new_tracker_l
new_target_domain_df

,seq,country,asn,target_domain,measurement_id,is_new_tracker
0,189728,se,2833,www.1177.se,43345322.0,False
1,186574,se,2119,www.1177.se,43340394.0,False
2,181448,se,42303,www.1177.se,43332830.0,False
3,181040,se,35100,www.1177.se,43332284.0,False
4,179460,se,51929,www.1177.se,43329833.0,False
...,...,...,...,...,...,...
494256,259407,fr,8362,a.zxcvads.com,43474978.0,True
494257,272167,fr,51207,a.zxcvads.com,43492070.0,True
494258,260201,fr,8218,a.zxcvads.com,43476138.0,True
494259,270236,fr,34970,a.zxcvads.com,43489380.0,True


# 8. Sanity Check

## 8.1. sanity_check_df

In [131]:
original_target_domain_fld_l = copy.deepcopy(target_domain_fld_l)

In [132]:
target_domain_fld_l[:10]

['instagram.com',
 'twitter.com',
 '2mdn.net',
 'clarity.ms',
 'filmix.ac',
 'quantserve.com',
 'gls-pakete.de',
 'telewebion.com',
 'n-tv.de',
 'media.net']

In [133]:
target_domain_fld_l = target_domain_df["target_domain_fld"]
print(target_domain_fld_l)

0             1177.se
1             1177.se
2             1177.se
3             1177.se
4             1177.se
             ...     
494256    zxcvads.com
494257    zxcvads.com
494258    zxcvads.com
494259    zxcvads.com
494260    zxcvads.com
Name: target_domain_fld, Length: 494261, dtype: object


In [134]:
sanity_check_df = pd.DataFrame({"target_domain_fld": target_domain_fld_l, "is_new_tracker": is_new_tracker_l})
sanity_check_df

,target_domain_fld,is_new_tracker
0,1177.se,False
1,1177.se,False
2,1177.se,False
3,1177.se,False
4,1177.se,False
...,...,...
494256,zxcvads.com,True
494257,zxcvads.com,True
494258,zxcvads.com,True
494259,zxcvads.com,True


## 8.2. FLD-level check

In [135]:
target_domain_fld_s = ""
inconsistent_target_domain_set = set()

for i in range(len(sanity_check_df)):
    if target_domain_fld_l[i] != target_domain_fld_s:
        target_domain_fld_s = target_domain_fld_l[i]
        is_new_tracker_bool = is_new_tracker_l[i]
    else:
        if is_new_tracker_l[i] != is_new_tracker_bool:
            inconsistent_target_domain_set.add(target_domain_fld_l[i])

# It should print an empty set.
print(inconsistent_target_domain_set)

set()


## 8.3. tracker_set-level check

In [156]:
should_be_all_true_l = list()

for problematic_fld_s in tracker_set:
    if problematic_fld_s in target_domain_fld_l:
        problematic_fld_count_i = target_domain_fld_l.count(problematic_fld_s)
        problematic_fld_index_i = target_domain_fld_l.index(problematic_fld_s)
        should_be_all_true_l.append(all(is_new_tracker_l[problematic_fld_index_i:problematic_fld_index_i+problematic_fld_count_i]))

# It should print True for all([True, True, True, ...]).
print(all(should_be_all_true_l))

True


## 8.4. Individual check

### 8.4.1. focus.de

In [136]:
problematic_fld_s = "focus.de"
print(problematic_fld_s in tracker_set)

problematic_fld_count_i = list(target_domain_fld_l).count(problematic_fld_s)
problematic_fld_index_i = list(target_domain_fld_l).index(problematic_fld_s)

print(sanity_check_df.iloc[problematic_fld_index_i:problematic_fld_index_i+problematic_fld_count_i])
print(all(is_new_tracker_l[problematic_fld_index_i:problematic_fld_index_i+problematic_fld_count_i]))

False
       target_domain_fld  is_new_tracker
145649          focus.de            True
145650          focus.de            True
145651          focus.de            True
145652          focus.de            True
145653          focus.de            True
...                  ...             ...
149056          focus.de            True
149057          focus.de            True
149058          focus.de            True
149059          focus.de            True
149060          focus.de            True

[3412 rows x 2 columns]
True


### 8.4.2. fwmrm.net

In [137]:
problematic_fld_s = "fwmrm.net"
print(problematic_fld_s in tracker_set)

problematic_fld_count_i = list(target_domain_fld_l).count(problematic_fld_s)
problematic_fld_index_i = list(target_domain_fld_l).index(problematic_fld_s)

print(sanity_check_df.iloc[problematic_fld_index_i:problematic_fld_index_i+problematic_fld_count_i])
print(all(is_new_tracker_l[problematic_fld_index_i:problematic_fld_index_i+problematic_fld_count_i]))

True
       target_domain_fld  is_new_tracker
151350         fwmrm.net            True
151351         fwmrm.net            True
151352         fwmrm.net            True
151353         fwmrm.net            True
151354         fwmrm.net            True
151355         fwmrm.net            True
151356         fwmrm.net            True
151357         fwmrm.net            True
151358         fwmrm.net            True
151359         fwmrm.net            True
151360         fwmrm.net            True
151361         fwmrm.net            True
151362         fwmrm.net            True
151363         fwmrm.net            True
151364         fwmrm.net            True
151365         fwmrm.net            True
151366         fwmrm.net            True
151367         fwmrm.net            True
151368         fwmrm.net            True
151369         fwmrm.net            True
151370         fwmrm.net            True
151371         fwmrm.net            True
True


### 8.4.3. instagram.com

In [151]:
problematic_fld_s = "instagram.com"
print(problematic_fld_s in tracker_set)

problematic_fld_count_i = list(target_domain_fld_l).count(problematic_fld_s)
problematic_fld_index_i = list(target_domain_fld_l).index(problematic_fld_s)

print(sanity_check_df.iloc[problematic_fld_index_i:problematic_fld_index_i+problematic_fld_count_i])
print(any(is_new_tracker_l[problematic_fld_index_i:problematic_fld_index_i+problematic_fld_count_i]))

False
       target_domain_fld  is_new_tracker
225195     instagram.com           False
225196     instagram.com           False
225197     instagram.com           False
225198     instagram.com           False
225199     instagram.com           False
...                  ...             ...
226860     instagram.com           False
226861     instagram.com           False
226862     instagram.com           False
226863     instagram.com           False
226864     instagram.com           False

[1670 rows x 2 columns]
False


In [152]:
any([False, False, False])

False

In [154]:
all([False, False, False])

False

### 8.4.4. twitter.com

In [158]:
problematic_fld_s = "twitter.com"
print(problematic_fld_s in tracker_set)

problematic_fld_count_i = list(target_domain_fld_l).count(problematic_fld_s)
problematic_fld_index_i = list(target_domain_fld_l).index(problematic_fld_s)

print(sanity_check_df.iloc[problematic_fld_index_i:problematic_fld_index_i+problematic_fld_count_i])
print(any(is_new_tracker_l[problematic_fld_index_i:problematic_fld_index_i+problematic_fld_count_i]))

False
       target_domain_fld  is_new_tracker
440618       twitter.com            True
440619       twitter.com            True
440620       twitter.com            True
440621       twitter.com            True
440622       twitter.com            True
...                  ...             ...
445904       twitter.com            True
445905       twitter.com            True
445906       twitter.com            True
445907       twitter.com            True
445908       twitter.com            True

[5291 rows x 2 columns]
True


# 9. Save as TXT

## 9.1. Save new_target_domain_df as TXT

In [122]:
os.chdir(output_path_s)
print(os.getcwd())

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task18_tracker_domain_tracker/output


In [148]:
new_target_domain_df = new_target_domain_df.sort_values(by=['seq'], ignore_index=True)
new_target_domain_df

,seq,country,asn,target_domain,measurement_id,is_new_tracker
0,0,de,39835,www.instagram.com,42506263.0,False
1,1,de,39835,twitter.com,42506264.0,True
2,2,de,39835,s0.2mdn.net,42506265.0,True
3,3,de,39835,www.clarity.ms,42506266.0,False
4,4,de,39835,www.filmix.ac,42506267.0,False
...,...,...,...,...,...,...
494256,494256,cz,44489,cdn.speedcurve.com,43832951.0,False
494257,494257,cz,44489,img.kurzy.cz,43832953.0,False
494258,494258,cz,44489,report.prozeny.cz,43832954.0,False
494259,494259,cz,44489,pro.mapy.cz,43832955.0,False


In [149]:
new_target_domain_df.to_csv("measurement_ids_kicho_tracker_20221013.txt", header=False, index=False)

## 9.2. Save new_tracker_target_domain_df as TXT

In [144]:
pd.DataFrame(set(new_tracker_target_domain_df["target_domain"])).to_csv("new_tracker_target_domain_20221013.txt", header=False, index=False)